In [ ]:
!pip install tensorflow==2.1.0


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
# import the necessary packages

from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import os
import PIL
#added
from tensorflow import keras
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation


In [ ]:
pip install h5py pyyaml

In [ ]:
!wget https://github.com/abcom-mltutorials/mask_detector/archive/master.zip

--2020-12-07 05:43:13--  https://github.com/abcom-mltutorials/mask_detector/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/abcom-mltutorials/mask_detector/zip/master [following]
--2020-12-07 05:43:13--  https://codeload.github.com/abcom-mltutorials/mask_detector/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [        <=>         ]  41.52M  27.6MB/s    in 1.5s    

2020-12-07 05:43:15 (27.6 MB/s) - ‘master.zip’ saved [43535909]



In [ ]:
! unzip master.zip

In [ ]:
#provide path directory and then mention both catogories for classification
path_dir = "/content/mask_detector-master/data"
categories = ["with_mask", "without_mask"]

In [ ]:
import io
import requests
from PIL import Image
#load the data with lbs
print("Loading the images")
# make arrays for storing dataset of images and it's label of 
# with mask and wihtout mask
dataset = []
lbs = []
# join path for preprocessing of images
for category in categories:
    path = os.path.join(path_dir, category)
    for img in os.listdir(path):
      img_path = os.path.join(path, img)
      print (img_path)
      Img = image.load_img(img_path, target_size=(224, 224))
      Img = image.img_to_array(Img)
      Img = preprocess_input(Img)
      
      dataset.append(Img)
      lbs.append(category)

print("All images loaded successfully")

In [ ]:
#now convert two categories into binary format like 0,1
label_binarizer = LabelBinarizer()
#Fit label binarizer and transform multi-class lbs to binary lbs.
lbs = label_binarizer.fit_transform(lbs)
#Converts a class vector (integers) to binary class matrix
lbs = to_categorical(lbs)

In [ ]:
#convert images to an array
dataset = np.array(dataset, dtype="float32")
lbs = np.array(lbs)

In [ ]:
#use label binarizer to split data for testing and training 
# i am using here test size of 20% and train size of 80%
#20% data will allocate to test set generally it is 20/80 or 30/70 for test/training ratio
# and random_state is for shuffling the data before using it to split between test and training set
(trainX, testX, trainY, testY) = train_test_split(dataset, lbs,
	test_size=0.20, stratify=lbs, random_state=42)

In [ ]:
#use augmentation to increase variation in dataset by various operation like shifting and rotating images
data_augmentation = image.ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.07, #0.15->0.07
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.10, #0.15->0.10
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# input_shape is shape of image(height, width, channeli.e. 3 here RGB)
# include_top is for fully connected layer
# imagenet is weights from pre-trained model
pre_trained_model = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=layers.Input(shape=(224, 224, 3)))

# we will freeze the base model to prevent them from updation while training
for layer in pre_trained_model.layers:
	layer.trainable = True

pre_trained_model.summary()

In [ ]:
#Save the last layer output
last_layer = pre_trained_model.get_layer("block_14_add")
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 7, 7, 160)


In [ ]:
#sequential model
model = tf.keras.Sequential(
    [keras.Input(shape=(224, 224, 3)),
     pre_trained_model,
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Dense(720, activation="relu"),
     layers.Dense(32, activation="relu"),
     layers.Dropout(0.3),
     layers.Flatten(),
     layers.Dense(units=2,activation="softmax")
     ] 
) 



In [ ]:
# Build the model
model.build([None, 224, 224, 3])
#print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 1280)        0         
_________________________________________________________________
dense (Dense)                (None, 3, 3, 720)         922320    
_________________________________________________________________
dense_1 (Dense)              (None, 3, 3, 32)          23072     
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 5

In [ ]:
# compile our model
from tensorflow.keras.optimizers import RMSprop
#initial_learning_rate = 1e-4
#optmzr=RMSprop(lr=initial_learning_rate, decay=initial_learning_rate / EPOCHS)
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              #optimizer=optmzr,
              metrics=["accuracy"])

In [ ]:
# Initialize values
EPOCHS =10
batchSize = 40

In [ ]:
# train the head of the network
FIT = model.fit(
	data_augmentation.flow(trainX, trainY, batch_size=batchSize),
	steps_per_epoch=len(trainX) // batchSize,
	validation_data=(testX, testY),
	validation_steps=len(testX) // batchSize,
	epochs= EPOCHS)


Epoch 1/10
85/85 [==============================] - 44s 518ms/step - loss: 0.1684 - accuracy: 0.9514 - val_loss: 0.7515 - val_accuracy: 0.7719
Epoch 2/10
85/85 [==============================] - 43s 507ms/step - loss: 0.0391 - accuracy: 0.9902 - val_loss: 1.2385 - val_accuracy: 0.8643
Epoch 3/10
85/85 [==============================] - 43s 504ms/step - loss: 0.0229 - accuracy: 0.9920 - val_loss: 1.9556 - val_accuracy: 0.8281
Epoch 4/10
85/85 [==============================] - 43s 509ms/step - loss: 0.0386 - accuracy: 0.9896 - val_loss: 0.4785 - val_accuracy: 0.9006
Epoch 5/10
85/85 [==============================] - 43s 503ms/step - loss: 0.0477 - accuracy: 0.9902 - val_loss: 0.0952 - val_accuracy: 0.9906
Epoch 6/10
85/85 [==============================] - 43s 502ms/step - loss: 0.0406 - accuracy: 0.9890 - val_loss: 0.4504 - val_accuracy: 0.9520
Epoch 7/10
85/85 [==============================] - 44s 520ms/step - loss: 0.0296 - accuracy: 0.9926 - val_loss: 1.4128 - val_accuracy: 0.8982

In [ ]:
# predictions on the testing set
PREDICT = model.predict(testX, batch_size=batchSize)

In [ ]:
# Make 2 lists to contain max values for predicted and actual test values
pred_class = []
ac_class = []
for i in range(len(PREDICT)):
    pr = PREDICT[i].argmax()
    pred_class.append(pr)
    ac = testY[i].argmax()
    ac_class.append(ac)

In [ ]:
# Make 2 lists to contain the predicted and actual output results gained from 
# comparing max values in last step with alotted strings     
output = {0:"with_mask", 1:"wihout_mask"}
pred_op = []
ac_op = []
for i in range(len(pred_class)):
   pred_op.append(output[pred_class[i]])
   ac_op.append(output[ac_class[i]])

In [ ]:
# Increment counter if the actual and predicted values match. Then calculate 
# % accuracy of the first batch.
correct_pred = 0
for i in range(batchSize):
    if pred_op[i] == ac_op[i]:
        correct_pred += 1

print("Accuracy of batch = ", (correct_pred/batchSize)*100, "%")

Accuracy of batch =  100.0 %


In [ ]:
#save model
model.save("mask_detector.model", save_format="h5")
print("The model is saved successfully!! Try detection by running the warn file.")

The model is saved successfully!! Try detection by running the warn file.


In [ ]:
print(tf.__version__)

2.3.0
